In [1]:
# copyright ############################### #
# This file is part of the Xtrack Package.  #
# Copyright (c) CERN, 2021.                 #
# ######################################### #
import ducktrack as dtk
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
from tqdm import tqdm

import xtrack as xt
import xobjects as xo
import xpart as xp


beta_rel = 0.305
#beta_rel = 0.106

gamma = 1.050
#gamma = 1.006

current=2.4
cooler_length = 1.5 # m cooler length
r_beam=25*1e-3

mass0=938.27208816*1e6 #ev/c^2

T_perp = 100e-3 # <E> [eV] = kb*T
T_l =  1e-3 # <E> [eV]
magnetic_field = 0.060 # T for AD
B_ratio=1e-4
Z=1

c=299792458.0

p0c = mass0*beta_rel*gamma #eV/c

circumference = 182.43280000000 #m
T = circumference/(c*beta_rel)
s_per_turn = T

beta_x=10 
beta_y=4

#disp_x=0.12
#disp_x=0
dx=0


import xobjects as xo
context = xo.ContextCpu()

num_particles=1
emitt_x=1*1e-6 #inital emittance

x_init=np.sqrt(beta_x*emitt_x)
px_init=np.sqrt(1/beta_x*emitt_x)
y_init=np.sqrt(beta_y*emitt_x)
py_init=np.sqrt(1/beta_y*emitt_x)
delta_init=5e-4

# Generate the normal distributions
# x_init = np.random.normal(loc=0, scale=x_init, size=num_particles)
# px_init = np.random.normal(loc=0, scale=px_init, size=num_particles)
# y_init = np.random.normal(loc=0, scale=y_init, size=num_particles)
# py_init = np.random.normal(loc=0, scale=py_init, size=num_particles)
#delta_init = np.random.normal(loc=0, scale=delta_init, size=num_particles)

In [2]:
gamma0 = 1.004469679
beta0 = np.sqrt(1 - 1 / gamma0**2)
mass0 = 193729.0248722061 * 1e6  # eV/c^2
clight = 299792458.0
p0c = mass0 * beta0 * gamma0  # eV/c
q0 = 54
particle_ref = xp.Particles(p0c=p0c,q0=q0,mass0=mass0,beta0=beta0,gamma0=gamma0)

circumference = 78.54370266  # m
T_per_turn = circumference/(clight*beta0)

qx = 1.82
qy = 2.72
beta_x = 5
beta_y = 5
qs=0.005247746218929317
bets0=-2078.673348423543

arc = xt.LineSegmentMap(
        qx=qx, qy=qy,
        length=circumference,
        betx=beta_x,
        bety=beta_y,
        )

arc_matching = xt.LineSegmentMap(
        qx=qx, qy=qy,
        length=circumference,
        betx=beta_x,
        bety=beta_y,
        qs=qs,
        bets=bets0)

line_matching=xt.Line([arc_matching])
line_matching.build_tracker()

num_particles=int(1e3)
sigma_dp = 5e-3    
gemitt_x = 14e-6
gemitt_y = 14e-6

nemitt_x = gemitt_x*beta0*gamma0
nemitt_y = gemitt_y*beta0*gamma0

particles = xp.generate_matched_gaussian_bunch(
        num_particles=num_particles,
        nemitt_x=nemitt_x, nemitt_y=nemitt_y, sigma_z=4.2,
        particle_ref=particle_ref,
        line=line_matching,        
        )

particles.delta = np.random.normal(loc=0.0, scale=sigma_dp, size=num_particles)
particles.zeta = np.random.uniform(-circumference/2, circumference/2, num_particles)
particles0=particles.copy()

max_time_s = 1
int_time_s = 1*1e-3
num_turns = int((max_time_s / T_per_turn).item())
save_interval = int((int_time_s / T_per_turn).item())

monitor = xt.ParticlesMonitor(start_at_turn=0, stop_at_turn=1,
                        n_repetitions=int(num_turns/save_interval),
                        repetition_period=save_interval,
                        num_particles=len(particles.x))

current = 0.6  # amperes
cooler_length = 2.5  # m cooler length
radius_e_beam = 25 * 1e-3
temp_perp = 100e-3 # <E> [eV] = kb*T
temp_long =  1e-3 # <E> [eV]
magnetic_field = 0.075  # T for LEIR

electron_cooler = xt.ElectronCooler(current=current,
                                length=cooler_length,
                                radius_e_beam=radius_e_beam,
                                temp_perp=temp_perp, temp_long=temp_long,
                                magnetic_field=magnetic_field)

line = xt.Line(elements=[monitor, electron_cooler, arc],element_names=['monitor','electron_cooler','arc'])                                    
line.particle_ref = particle_ref
context = xo.ContextCpu(omp_num_threads=4)
line.build_tracker(_context=context)

particles=particles0.copy()

record = line.start_internal_logging_for_elements_of_type(
                                                    xt.ElectronCooler, capacity=10000)

line.track(particles, num_turns=num_turns,
        turn_by_turn_monitor=False,with_progress=True)

Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.
Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.


Tracking:   0%|          | 0/359700 [00:00<?, ?it/s]

In [3]:
line.track(particles, num_turns=num_turns,
                        turn_by_turn_monitor=False,with_progress=True)


# extract relevant values
x_xs = monitor.x[:,:,0]
px_xs = monitor.px[:,:,0]
y_xs = monitor.y[:,:,0]
py_xs = monitor.py[:,:,0]
delta_xs = monitor.delta[:,:,0]
zeta_xs = monitor.zeta[:,:,0]
#time_xs = monitor.at_turn[:, 0, 0] * T_per_turn

# compute actions. for x, remove the dp/p contribution:
action_x = ((x_xs-dx*delta_xs)**2/beta_x + beta_x*px_xs**2)
# for y, simple compute:
action_y = (y_xs**2/beta_y + beta_y*py_xs**2)
emittance_x=np.mean(action_x, axis=1)/2
# norm_emittance_x=np.mean(action_x,axis=1)/2*gamma0*beta0
# norm_emittance_y=np.mean(action_y,axis=1)/2*gamma0*beta0

geo_emittance_x=np.mean(action_x,axis=1)/2
geo_emittance_y=np.mean(action_y,axis=1)/2

Tracking:   0%|          | 0/359700 [00:00<?, ?it/s]

In [6]:
record.Fl

array([-3.17508624e+10,  1.65303706e+10, -4.67435512e+10, ...,
        5.54469960e+09,  2.85147198e+10,  6.28273484e+09])

In [5]:
import ducktrack as dtk
dtk_particle = dtk.TestParticles(
        
        mass0=mass0,
        p0c=p0c,
        x=x_init,
        px=px_init,
        y=y_init,
        py=py_init,
        delta=delta_init,
        zeta=0)

dtk_cooler = dtk.elements.ElectronCooler(current=current,length=cooler_length,radius_e_beam=r_beam,
                                temp_perp=T_perp,temp_long=T_l,
                                magnetic_field=magnetic_field,magnetic_field_ratio=B_ratio,
                                space_charge_factor=1,
                                offset_energy=10)
# initialize arrays
x = []
px = []
action_x = []
action_y = []
py = []
delta = []

for i in tqdm(range(num_turns)):
    
    x.append(dtk_particle.x)
    px.append(dtk_particle.px) 
    py.append(dtk_particle.py)
    delta.append(dtk_particle.delta)

    # calculate action in horizontal plane
    action_x_temp = (dtk_particle.x**2/beta_x + beta_x*dtk_particle.px**2)
    action_x.append(action_x_temp)
    action_y_temp = (dtk_particle.y**2/beta_y + beta_y*dtk_particle.py**2)
    action_y.append(action_y_temp)

    #arc.track(dtk_particle)
    dtk_cooler.track(dtk_particle)

# convert arrays to numpy arrays
x = np.array(x)
px = np.array(px)
action_x = np.array(action_x)

time = np.arange(0, num_turns, step) * s_per_turn


100%|██████████| 359675/359675 [00:37<00:00, 9601.14it/s] 


NameError: name 'step' is not defined

In [ ]:
# Plot the horizontal action versus time
plt.figure()

plt.plot(time,px,label='ducktrack')
plt.plot(time,px_xs.flatten(),label='Xsuite')

plt.xlabel('Time [s]')
plt.ylabel('Px')
plt.title('PX')
plt.legend()
plt.ticklabel_format(useOffset=False)

In [ ]:
particles.beta0

In [ ]:
# Plot the horizontal action versus time
plt.figure()

plt.plot(time,py,label='ducktrack')
plt.plot(time,py_xs,label='Xsuite')

plt.xlabel('Time [ms]')
plt.ylabel('PY [rad]')
plt.title('PY')
plt.legend()
plt.ticklabel_format(useOffset=False)

In [ ]:
# Plot the horizontal action versus time
plt.figure()

plt.plot(time,delta,label='ducktrack')
plt.plot(time,delta_xs,label='Xsuite')

plt.xlabel('Time [ms]')
plt.ylabel('Delta []')
plt.title('Delta')
plt.legend()
plt.ticklabel_format(useOffset=False)

In [23]:
# import xobjects as xo
# test_context=xo.ContextCpu()

# assert np.isclose(test_context.nparray_from_context_array(particles.x)[0],
#                     dtk_particle.x, rtol=1e-14, atol=1e-14)
# assert np.isclose(test_context.nparray_from_context_array(particles.px)[0],
#                     dtk_particle.px, rtol=1e-14, atol=1e-14)
# assert np.isclose(test_context.nparray_from_context_array(particles.y)[0],
#                     dtk_particle.y, rtol=1e-14, atol=1e-14)
# assert np.isclose(test_context.nparray_from_context_array(particles.py)[0],
#                     dtk_particle.py, rtol=1e-14, atol=1e-14)
# assert np.isclose(test_context.nparray_from_context_array(particles.zeta)[0],
#                     dtk_particle.zeta, rtol=1e-14, atol=1e-14)
# assert np.isclose(test_context.nparray_from_context_array(particles.delta)[0],
#                     dtk_particle.delta, rtol=1e-14, atol=1e-14)

In [24]:
# dtk_particle = dtk.TestParticles(
#         mass0=mass0,
#         p0c=p0c,
#         x=x_init,
#         px=px_init,
#         y=y_init,
#         py=py_init,
#         delta=delta_init,
#         zeta=0)



# particles = xp.Particles.from_dict(dtk_particle.to_dict(),
#                                     _context=test_context)
# I=2.4
# L = 1.5 # m cooler length
# r_beam=25*1e-3

# T_perp = 0.01 # <E> [eV] = kb*T
# T_l =  0.001 # <E> [eV]
# B = 0.060 # T for AD
# B_ratio=1e-3
# Z=0
# electron_cooler=xt.ElectronCooler(current=current,length=cooler_length,r_beam=r_beam,
#                                                 T_perp=T_perp,T_l=T_l,
#                                                 magnetic_field=magnetic_field,B_ratio=B_ratio,
#                                                 Neutralisation_space_charge=1,Neutralisation_rotation=1)

# electron_cooler.track(particles)

# dtk_electron_cooler = dtk.elements.ElectronCooler(current=current,length=cooler_length,r_beam=r_beam,
#                                                 T_perp=T_perp,T_l=T_l,
#                                                 magnetic_field=magnetic_field,B_ratio=B_ratio,
#                                                 Neutralisation_space_charge=1,Neutralisation_rotation=1)

# dtk_electron_cooler.track(dtk_particle)


# print(dtk_particle.px)
# print(particles.px[0])


# assert np.isclose(test_context.nparray_from_context_array(particles.x)[0],
#                     dtk_particle.x, rtol=1e-14, atol=1e-14)
# assert np.isclose(test_context.nparray_from_context_array(particles.px)[0],
#                     dtk_particle.px, rtol=1e-14, atol=1e-14)
# assert np.isclose(test_context.nparray_from_context_array(particles.y)[0],
#                     dtk_particle.y, rtol=1e-14, atol=1e-14)
# assert np.isclose(test_context.nparray_from_context_array(particles.py)[0],
#                     dtk_particle.py, rtol=1e-14, atol=1e-14)
# assert np.isclose(test_context.nparray_from_context_array(particles.zeta)[0],
#                     dtk_particle.zeta, rtol=1e-14, atol=1e-14)
# assert np.isclose(test_context.nparray_from_context_array(particles.delta)[0],
#                     dtk_particle.delta, rtol=1e-14, atol=1e-14)


